In [4]:
import pathlib
import jinja2
from xml.dom import minidom
from tqdm import tqdm
from IPython.display import HTML
import re


In [11]:
def clean_svg(path):
    svg_keep = {'viewBox'}
    doc = minidom.parse(str(path)).documentElement
    clean_attrs(doc, None, {'viewBox'})
    clean_children(doc)
    return doc

def clean_children(element):
    element_remove = {'stroke', 'style', 'stroke-linejoin', 'stroke-miterlimit', 'fill', 'stroke-width'}
    for child in list(element.childNodes):
        if child.nodeType != child.ELEMENT_NODE:
            element.removeChild(child)
        else:
            clean_attrs(child, element_remove)
            clean_children(child)
        
def clean_attrs(child, remove, keep=None):
    for attr in list(child.attributes.keys()):
        if remove and attr in remove:
            child.attributes.removeNamedItem(attr)
        elif keep and not attr in keep:
            child.attributes.removeNamedItem(attr)
        
def pascal_case(name):
    return re.sub(r'[_\W]', '', name.title())

def main():
    readdir = pathlib.Path.home() / 'Desktop' / 'icons'
    writedir = pathlib.Path('.') / 'icons'
    writedir.mkdir(exist_ok=True)
    style = 'fill:none; stroke:black; stroke-width:2;'
    html_header = '<style>svg{' + style + ' width: 3em;}</style>'
    xml_header = '\n'.join([
    '<?xml version="1.0" encoding="utf-8"?>',
    ])
    icons = [icon for icon in readdir.iterdir() if icon.suffix == '.svg']

    def wrap(title, tag):
        return '<span title="{0}">{1}</span>'.format(title, tag)

    svgs = []
    for icon in tqdm(sorted(icons)):
        svg = clean_svg(icon)
        make_component(svg, icon.stem)
        svg.attributes['id'] = icon.stem
        svg.attributes['class'] = 'icon'
        svg.attributes['style'] = style
        svgs.append('<span title={}>{}</span>'.format(
            pascal_case(icon.stem), svg.toxml()
        ))
#         writeto = writedir / icon.name
#         writeto.write_text(xml_header+svg)

    html = html_header + ''.join(svgs)
    return HTML(html)

template = """
import React from 'react'

const {{ name }} = () => (
  {{ xml|indent(2) }}
)
export default {{ name }}
""".strip()

def make_component(svg_node, name):
    componentdir = pathlib.Path('.') / '..' / 'components' / 'lineicons'
    componentdir.mkdir(exist_ok=True)
    componentdir = componentdir.resolve()
    clone = svg_node.cloneNode(True)
    clone.attributes['className'] = 'Icon'
    component_name = pascal_case(name)
    clone.attributes['id'] = component_name
    tmp = jinja2.Template(template)
    xml = clone.toprettyxml('  ').replace('/>', ' />')
    result = tmp.render(name=component_name, xml=xml)
    component_path = (componentdir / component_name).with_suffix('.js')
    component_path.write_text(result + '\n')
    
main()



100%|██████████| 715/715 [00:06<00:00, 80.36it/s]


In [400]:
def significant_digits(num, digits=3):
    return '{num:.{digits}f}'.format(num=float(num), digits=digits)

significant_digits('213.23')

'213.230'

In [410]:
import re
data = {}
for path in dom.getElementsByTagName('path'):
    path_d = path.attributes['d'].value
    icon_id = path.attributes['id'].value
    path_d = re.sub(r'-?\d+\.\d+', lambda m: significant_digits(m.group(), 3), path_d)
    data[icon_id.lower()] = path_d

data

{'codepen': 'm 4.428,23.059 27.885,18.577 0,18.577 L 4.428,41.636 m 27.885,0 27.885,-18.577 0,18.577 -27.885,18.577 m 27.885,-18.577 -27.885,-18.577 0,-18.577 27.885,18.577 m -27.885,0 -27.885,18.577 0,-18.577 L 32.313,4.482',
 'face-monogram': 'm 10.833,16.877 c -0.036,-2.862 2.141,-4.665 5.435,-4.076 3.294,0.589 4.047,4.759 1.882,6.950 -2.941,2.945 -9.364,2.945 -12.234,-0.707 -5.141,-6.361 1.494,-16.256 10.940,-15.432 6.329,0.568 11.034,7.386 10.576,13.665 -0.314,4.241 -1.859,8.788 -4.000,12.958 -2.576,5.018 -7.176,9.471 -12.940,9.659 -3.294,0.107 -6.894,-2.356 -6.352,-5.890 0.699,-4.594 5.764,-6.997 9.999,-7.421 8.235,-0.821 16.469,0.941 24.704,-0.019 7.105,-0.832 16.587,-3.934 20.469,-11.273 1.682,-3.192 2.400,-7.197 -1.039,-9.930 -3.423,-2.721 -9.329,-0.954 -12.352,1.178 -5.188,3.687 -7.788,10.354 -9.646,16.492 -3.200,10.590 -6.235,21.557 -12.940,30.627 -2.823,3.840 -7.646,7.292 -12.940,6.125 -3.176,-0.699 -6.823,-5.207 -5.964,-8.269 1.188,-4.241 4.823,-5.030 8.940,-5.478 6.588,-0

In [386]:
icon_id.value

'face-monogram'

In [431]:
ldir = pathlib.Path('..') / 'components' / 'lineicons'
icons = [i.stem for i in ldir.iterdir()]
exports = '\n'.join('export {{ {0} }} from "./{0}.js"'.format(icon) for icon in icons)
index = ldir / 'index.js'
index.write_text(exports)

45440